### Write a Python (or Java) program to implement a Content-based Recommender System that takes a movie name as input and recommends top-3 movies to the user. Use genres as features and the provided ratings for vectorization of all profiles (refer to Lecture 3 slides for context).

In [11]:
import pandas as pd
import numpy as np
import math 
import matplotlib 
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")
import nltk

from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies = pd.read_csv(r"D:\FastSemesters\semester6\Recommender_System(E)\A1\ml-latest-small\movies.csv")

In [3]:
movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [25]:
data = pd.DataFrame(movies)
data.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [26]:
data['genres'][0]

'Adventure|Animation|Children|Comedy|Fantasy'

In [27]:
data['genres'] = data.genres.str.split('|')
data['genres'] = data['genres'].apply(lambda x : str(x).lower())
data['year'] = data.title.str.extract('(\(\d\d\d\d\))',expand=False)
data['year'] = data.title.str.extract('(\d\d\d\d)',expand=False)
data['title'] = data.title.str.replace('(\(\d\d\d\d\))', '')
data['title'] = data['title'].apply(lambda x: x.strip())
data.head(5)

,movieId,title,genres,year
0,1,Toy Story,"['adventure', 'animation', 'children', 'comedy...",1995
1,2,Jumanji,"['adventure', 'children', 'fantasy']",1995
2,3,Grumpier Old Men,"['comedy', 'romance']",1995
3,4,Waiting to Exhale,"['comedy', 'drama', 'romance']",1995
4,5,Father of the Bride Part II,['comedy'],1995


In [28]:
cv = CountVectorizer(max_features = 4000, stop_words = 'english')
oneHotEncoder = cv.fit_transform(movies['genres']).toarray()

In [29]:
print(oneHotEncoder)

[[0 1 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [30]:
similarity = cosine_similarity(oneHotEncoder)
similarity

array([[1.        , 0.77459667, 0.31622777, ..., 0.        , 0.31622777,
        0.4472136 ],
       [0.77459667, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31622777, 0.        , 1.        , ..., 0.        , 0.        ,
        0.70710678],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.31622777, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.4472136 , 0.        , 0.70710678, ..., 0.        , 0.        ,
        1.        ]])

In [31]:
def CBRS(title, data, similarity):
    
    index = np.where(data['title'] == title)[0][0]
    similar_movies = sorted(enumerate(similarity[index]), key=lambda x: x[1], reverse=True)[1:4]
    recommended = []
    for i in similar_movies:
        recommended.append(movies['title'][i[0]])
        
    return recommended

In [34]:
CBRS('Waiting to Exhale', data, similarity)

['American President, The (1995)',
 'Mighty Aphrodite (1995)',
 'Postman, The (Postino, Il) (1994)']

In [35]:
CBRS('Father of the Bride Part II', data, similarity)

['Four Rooms (1995)',
 'Ace Ventura: When Nature Calls (1995)',
 'Bio-Dome (1996)']

In [36]:
CBRS('Jumanji', data, similarity)

['Indian in the Cupboard, The (1995)',
 'NeverEnding Story III, The (1994)',
 'Escape to Witch Mountain (1975)']